In [32]:
import pandas as pd
import numpy as np
import xarray as xr
import calliope
calliope.set_log_verbosity('INFO', include_solver_output=True)

In [2]:
# Load your weather data
df = pd.read_csv("df_KNMI.csv",index_col=0)

In [3]:
df.sample(5)

,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10N,TD,Q,SQ,VV,N,U,P
500,330,20240121,21,190,160,160,220,70,NaN,51,0,0,NaN,NaN,88,10098
573,330,20240124,22,270,90,100,130,85,NaN,54,0,0,NaN,NaN,80,10276
526,330,20240122,23,250,120,120,180,83,NaN,39,0,0,NaN,NaN,74,10150
160,330,20240107,17,60,80,80,110,-3,NaN,-50,0,0,NaN,NaN,70,10288
276,330,20240112,13,10,60,50,90,63,NaN,32,26,0,NaN,NaN,80,10334


###### df.head()

In [4]:
# Ensure 'HH' is properly formatted
df["HH"] = df["HH"].astype(str).str.zfill(2)

In [5]:
# Handle '24' hour values by shifting to next day
def parse_datetime(row):
    hour = int(row["HH"])
    date = pd.to_datetime(str(row["YYYYMMDD"]), format="%Y%m%d")
    if hour == 24:
        return date + pd.Timedelta(days=1)
    else:
        return date + pd.Timedelta(hours=hour)

df["datetime"] = df.apply(parse_datetime, axis=1)
df.set_index("datetime", inplace=True)

In [6]:
# New column names 
new_column_names = { 
    'FH' : 'average windspeed (m/s)',
    'FF' : 'windspeed previous 10 minuten (m/s)',
    'FX' : 'highest windspeed (m/s)',
    'T'  : 'temperature_C',
    'SQ' : 'duration of sunshine (h)',
    'Q'  : 'global_radiation_J/m2',    
    'P'  : 'air pressure (Pa)',
    'N'  : 'cloud cover (-)',
    'U'  : 'relative humidity (in percentages)'
}
df = df.rename(columns = new_column_names)


In [7]:
df.sample(5)

,STN,YYYYMMDD,HH,DD,average windspeed (m/s),windspeed previous 10 minuten (m/s),highest windspeed (m/s),temperature_C,T10N,TD,global_radiation_J/m2,duration of sunshine (h),VV,cloud cover (-),relative humidity (in percentages),air pressure (Pa)
datetime,,,,,,,,,,,,,,,,
2024-01-13 00:00:00,330,20240112,24,280,30,30,50,50,8.0,45,0,0,NaN,NaN,96,10293
2024-01-17 03:00:00,330,20240117,03,200,60,70,90,-3,NaN,-30,0,0,NaN,NaN,81,9983
2024-01-17 01:00:00,330,20240117,01,190,70,70,100,2,NaN,-28,0,0,NaN,NaN,79,9998
2024-01-14 12:00:00,330,20240114,12,270,90,80,120,58,45.0,7,21,0,NaN,NaN,69,10093
2024-01-04 00:00:00,330,20240103,24,280,90,90,120,89,75.0,74,0,0,NaN,NaN,89,9956


In [8]:
drop_columns = ['YYYYMMDD', 'HH', 'DD', 'T10N', 'TD', 'VV', 'STN']
df = df.drop(columns = drop_columns)

In [16]:
irradiance_df=pd.read_csv('data_experimenting/df_with_calculations.csv',index_col=0)

In [17]:
irradiance_df.sample(5)

,YYYYMMDDHH,average windspeed (m/s),windspeed previous 10 minuten (m/s),highest windspeed (m/s),temperature in degrees Celcius,global radiation (in J/m^2),duration of sunshine (h),cloud cover (-),relative humidity (in percentages),air pressure (Pa),yield solar panel (kWh),Plane-of-Array_irradiance
101,2024-01-05 06:00:00,100,110,140,78,0.0,0.0,NaN,78,9924,0.0,0.0
342,2024-01-15 07:00:00,130,140,240,29,0.0,0.0,NaN,85,10022,0.0,0.0
527,2024-01-23 00:00:00,140,150,200,78,0.0,0.0,NaN,76,10161,0.0,0.0
702,2024-01-30 07:00:00,80,80,110,107,0.0,0.0,NaN,82,10237,0.0,0.0
219,2024-01-10 04:00:00,80,80,100,-40,0.0,0.0,NaN,69,10305,0.0,0.0


In [18]:
energy_demand=((irradiance_df["Plane-of-Array_irradiance"] * 3) / 4) * -1

In [19]:
energy_demand.sample(5)

315   -0.000000
188   -0.000000
158   -0.012138
183   -0.006073
6     -0.000000
Name: Plane-of-Array_irradiance, dtype: float64

In [20]:
df["energy_demand"]= energy_demand.values

In [23]:
poa_irradiance= irradiance_df["Plane-of-Array_irradiance"]

In [24]:
df['poa_irradiance'] =poa_irradiance.values

In [25]:
df.sample(5)

,average windspeed (m/s),windspeed previous 10 minuten (m/s),highest windspeed (m/s),temperature_C,global_radiation_J/m2,duration of sunshine (h),cloud cover (-),relative humidity (in percentages),air pressure (Pa),energy_demand,poa_irradiance
datetime,,,,,,,,,,,
2024-01-01 13:00:00,120,120,160,90,51,3,NaN,71,10035,-0.117857,0.157142
2024-01-15 12:00:00,120,120,170,33,55,3,NaN,80,10035,-0.151114,0.201486
2024-01-17 21:00:00,60,50,70,24,0,0,NaN,76,9920,-0.000000,0.000000
2024-01-04 05:00:00,80,90,120,90,0,0,NaN,89,9995,-0.000000,0.000000
2024-01-05 16:00:00,50,40,60,84,1,0,NaN,85,9980,-0.002024,0.002699


In [26]:
# Keep only relevant columns
df_clean = df[["poa_irradiance", "energy_demand"]]

In [27]:
# Reset index to make datetime a column
df_clean = df_clean.reset_index()

# Sort by datetime for clarity (optional)
df_clean = df_clean.sort_values("datetime")

In [28]:
# Save to CSV in correct format
df_clean.to_csv("solar_model/time_series/pv_resource.csv", index=False)

In [33]:
model=calliope.Model('weatherdata_solar_model/model.yaml')

[2025-06-09 00:42:30] INFO     Model: initialising
[2025-06-09 00:42:30] INFO     Model: preprocessing stage 1 (model_run)
[2025-06-09 00:42:32] INFO     Model: preprocessing stage 2 (model_data)
[2025-06-09 00:42:32] WARNING  C:\Users\csten\anaconda3\envs\calliope\lib\site-packages\calliope\preprocess\checks.py:767: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


[2025-06-09 00:42:32] WARNING  C:\Users\csten\anaconda3\envs\calliope\lib\site-packages\calliope\exceptions.py:60: ModelWarning:

Possible issues found during model processing:
 * Operational mode requires the same timestep resolution profile to be emulated on each date


[2025-06-09 00:42:32] INFO     Model: preprocessing complete


In [34]:
model.inputs

<xarray.Dataset>
Dimensions:                       (loc_techs_finite_resource: 2,
                                   loc_techs_area: 1, loc_techs: 2,
                                   loc_techs_export: 1, timesteps: 744,
                                   costs: 1, loc_techs_investment_cost: 1,
                                   techs: 2, loc_carriers: 1,
                                   loc_techs_non_conversion: 2, locs: 1,
                                   carriers: 1)
Coordinates:
  * carriers                      (carriers) <U11 'electricity'
  * costs                         (costs) object 'monetary'
  * loc_carriers                  (loc_carriers) object 'mijn_stad::electricity'
  * loc_techs                     (loc_techs) object 'mijn_stad::pv' 'mijn_st...
  * loc_techs_area                (loc_techs_area) object 'mijn_stad::pv'
  * loc_techs_export              (loc_techs_export) object 'mijn_stad::pv'
  * loc_techs_finite_resource     (loc_techs_finite_resource) object 'mijn_st...
  * loc_techs_investment_cost     (loc_techs_investment_cost) object 'mijn_st...
  * loc_techs_non_conversion      (loc_techs_non_conversion) object 'mijn_sta...
  * locs                          (locs) <U9 'mijn_stad'
  * techs                         (techs) object 'demand_electricity' 'pv'
  * timesteps                     (timesteps) datetime64[ns] 2024-01-01T01:00...
Data variables: (12/22)
    resource_eff                  (loc_techs_finite_resource) float64 1.0 nan
    resource_area_per_energy_cap  (loc_techs_area) int32 8
    energy_cap_max                (loc_techs) float64 10.0 nan
    force_resource                (loc_techs_finite_resource) float64 nan 1.0
    energy_con                    (loc_techs) float64 nan 1.0
    export_carrier                (loc_techs_export) object 'electricity'
    ...                            ...
    lookup_loc_techs              (loc_techs_non_conversion) object 'mijn_sta...
    lookup_loc_techs_export       (loc_techs_export) object 'mijn_stad::pv::e...
    lookup_loc_techs_area         (locs) object 'mijn_stad::pv'
    timestep_resolution           (timesteps) float64 1.0 1.0 1.0 ... 1.0 1.0
    timestep_weights              (timesteps) float64 1.0 1.0 1.0 ... 1.0 1.0
    max_demand_timesteps          (carriers) datetime64[ns] 2024-01-27T11:00:00
Attributes:
    calliope_version:    0.6.10
    applied_overrides:   
    scenario:            None
    defaults:            carrier_ratios:\ncharge_rate:\nenergy_cap_per_storag...
    allow_operate_mode:  0

In [35]:
# Individual data variables can be accessed easily, `to_pandas()` reformats the data to look nicer
model.inputs.resource.to_pandas()

timesteps,2024-01-01 01:00:00,2024-01-01 02:00:00,2024-01-01 03:00:00,2024-01-01 04:00:00,2024-01-01 05:00:00,2024-01-01 06:00:00,2024-01-01 07:00:00,2024-01-01 08:00:00,2024-01-01 09:00:00,2024-01-01 10:00:00,...,2024-01-31 15:00:00,2024-01-31 16:00:00,2024-01-31 17:00:00,2024-01-31 18:00:00,2024-01-31 19:00:00,2024-01-31 20:00:00,2024-01-31 21:00:00,2024-01-31 22:00:00,2024-01-31 23:00:00,2024-02-01 00:00:00
loc_techs_finite_resource,,,,,,,,,,,,,,,,,,,,,
mijn_stad::pv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026207,0.106999,...,0.042252,0.032389,0.002699,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mijn_stad::demand_electricity,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.019655,-0.080249,...,-0.031689,-0.024291,-0.002024,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


In [38]:
model.get_formatted_array('resource').sum('locs').to_pandas()

timesteps,2024-01-01 01:00:00,2024-01-01 02:00:00,2024-01-01 03:00:00,2024-01-01 04:00:00,2024-01-01 05:00:00,2024-01-01 06:00:00,2024-01-01 07:00:00,2024-01-01 08:00:00,2024-01-01 09:00:00,2024-01-01 10:00:00,...,2024-01-31 15:00:00,2024-01-31 16:00:00,2024-01-31 17:00:00,2024-01-31 18:00:00,2024-01-31 19:00:00,2024-01-31 20:00:00,2024-01-31 21:00:00,2024-01-31 22:00:00,2024-01-31 23:00:00,2024-02-01 00:00:00
techs,,,,,,,,,,,,,,,,,,,,,
demand_electricity,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.019655,-0.080249,...,-0.031689,-0.024291,-0.002024,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026207,0.106999,...,0.042252,0.032389,0.002699,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
model.run()

[2025-06-09 00:49:01] INFO     Backend: starting model run
[2025-06-09 00:49:01] INFO     Loading sets
[2025-06-09 00:49:01] INFO     Loading parameters
[2025-06-09 00:49:02] INFO     constraints are loaded in the following order: ['capacity', 'costs', 'dispatch', 'energy_balance', 'network', 'policy', 'conversion', 'conversion_plus', 'group', 'export', 'milp']
[2025-06-09 00:49:02] INFO     creating capacity constraints
[2025-06-09 00:49:02] INFO     creating costs constraints
[2025-06-09 00:49:02] INFO     creating dispatch constraints
[2025-06-09 00:49:02] INFO     creating energy_balance constraints
[2025-06-09 00:49:03] INFO     creating network constraints
[2025-06-09 00:49:03] INFO     creating policy constraints
[2025-06-09 00:49:03] INFO     creating conversion constraints
[2025-06-09 00:49:03] INFO     creating conversion_plus constraints
[2025-06-09 00:49:03] INFO     creating group constraints
[2025-06-09 00:49:03] INFO     creating export constraints
[2025-06-09 00:49:03] 

In [37]:
model.results

<xarray.Dataset>
Dimensions:                                 (loc_techs: 2,
                                             loc_tech_carriers_prod: 1,
                                             timesteps: 744,
                                             loc_tech_carriers_con: 1,
                                             costs: 1, loc_techs_cost: 1,
                                             loc_techs_area: 1,
                                             ...
                                             loc_tech_carriers_export: 1,
                                             loc_techs_investment_cost: 1,
                                             loc_techs_cost_investment_constraint: 1,
                                             loc_carriers_system_balance_constraint: 1,
                                             loc_techs_balance_demand_constraint: 1,
                                             carriers: 1, techs: 2)
Coordinates: (12/15)
  * techs                                   (techs) object 'demand_electricit...
  * costs                                   (costs) <U8 'monetary'
  * loc_carriers_system_balance_constraint  (loc_carriers_system_balance_constraint) <U22 ...
  * loc_tech_carriers_con                   (loc_tech_carriers_con) <U42 'mij...
  * loc_tech_carriers_export                (loc_tech_carriers_export) <U26 '...
  * loc_tech_carriers_prod                  (loc_tech_carriers_prod) <U26 'mi...
    ...                                      ...
  * loc_techs_cost                          (loc_techs_cost) <U13 'mijn_stad:...
  * loc_techs_cost_investment_constraint    (loc_techs_cost_investment_constraint) <U13 ...
  * loc_techs_investment_cost               (loc_techs_investment_cost) <U13 ...
  * loc_techs_supply_plus                   (loc_techs_supply_plus) <U13 'mij...
  * timesteps                               (timesteps) datetime64[ns] 2024-0...
  * carriers                                (carriers) <U11 'electricity'
Data variables: (12/16)
    energy_cap                              (loc_techs) float64 0.2941 0.2941
    carrier_prod                            (loc_tech_carriers_prod, timesteps) float64 ...
    carrier_con                             (loc_tech_carriers_con, timesteps) float64 ...
    cost                                    (costs, loc_techs_cost) float64 3...
    resource_area                           (loc_techs_area) float64 2.353
    resource_con                            (loc_techs_supply_plus, timesteps) float64 ...
    ...                                      ...
    system_balance                          (loc_carriers_system_balance_constraint, timesteps) float64 ...
    required_resource                       (loc_techs_balance_demand_constraint, timesteps) float64 ...
    capacity_factor                         (timesteps, loc_tech_carriers_prod) float64 ...
    systemwide_capacity_factor              (carriers, techs) float64 nan 0.1022
    systemwide_levelised_cost               (carriers, costs, techs) float64 ...
    total_levelised_cost                    (carriers, costs) float64 0.1662
Attributes:
    termination_condition:     optimal
    objective_function_value:  3.7148649
    solution_time:             4.736482
    time_finished:             2025-06-09 00:49:06
    calliope_version:          0.6.10
    applied_overrides:         
    scenario:                  None
    defaults:                  carrier_ratios:\ncharge_rate:\nenergy_cap_per_...
    allow_operate_mode:        0
    model_config:              calliope_version: 0.6.10\nname: Weather-model\...
    run_config:                backend: pyomo\nbigM: 1000000000.0\ncyclic_sto...

### Model Analysis

In [42]:
model.plot.capacity()

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

###  End Calliope

In [ ]:
# Convert raw values
df["temperature_C"] = df["temperature_C"] / 10
df["global_radiation_KW"] = (df["global_radiation_KW"] * 1e4) / (1e3 * 3600 )
df["duration of sunshine (h)"] = df["duration of sunshine (h)"] * 0.1

In [ ]:
df_clean.sample(5)

In [ ]:
# Constants
area_of_solar_panel = 2.47  # in m^2
efficiency_solar_panel = 0.17  # unitless efficiency
number_of_solar_panels = 1  # if multiple, change this

# Calculate irradiance in kWh for each hour
irradiance_kwh = (df['air pressure (Pa)'] * area_of_solar_panel * number_of_solar_panels * efficiency_solar_panel) / (1e3 * 3600)
irradiance_kwh.name = "nl_site"  # Important for Calliope

# Save to CSV
irradiance_kwh.to_csv("solar_model/time_series/irradiance.csv", header=True)


In [ ]:
# Save CSV with location name as header
irradiance_kwh.to_csv("solar_model/time_series/irradiance.csv", header=["nl_site"])



In [ ]:
pd.read_csv("solar_model/time_series/irradiance.csv", header=None).head()

In [ ]:
# Check row counts per day
daily_counts = df.groupby(df.index.date).size()
print(daily_counts.value_counts())  # Should show 24s only

In [ ]:
# To find problematic days
print(daily_counts[daily_counts != 24])


In [ ]:
# Load and inspect the CSV
df = pd.read_csv("solar_model/time_series/irradiance.csv")

# Display first few rows and column names
print(df.columns)
print(df.head())

In [ ]:
model = calliope.Model("solar_model/model.yaml")
